In [ ]:
!pip install transformers
!pip install torch

In [ ]:
# Import libraries
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/NLP Project/final_factuality_model"))

In [ ]:
# Replace with the path where your model is saved
model_path = "/content/drive/MyDrive/NLP Project/final_factuality_model"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the complete dataset
df = pd.read_csv("/content/drive/MyDrive/NLP Project/preprocessed_dataset_final1.csv")

# Split into 80% train and 20% test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Check the size of train and test splits
print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

In [ ]:
test_df['processed_claims'] = test_df['processed_claims'].astype(str)
test_df['processed_evidence'] = test_df['processed_evidence'].astype(str)

In [ ]:
def predict_claim_evidence(claim, evidence, threshold=0.5):
    inputs = tokenizer(claim, evidence, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_class = 1 if probabilities[0][1] > threshold else 0
    return predicted_class

In [ ]:
test_df['predicted_label'] = test_df.apply(lambda row: predict_claim_evidence(row['processed_claims'], row['processed_evidence']), axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Calculate metrics without displaying them
true_labels = test_df['label'].tolist()
predicted_labels = test_df['predicted_label'].tolist()

In [ ]:
accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='binary')

In [ ]:
performance_metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1
}

In [ ]:
import matplotlib.pyplot as plt

def plot_comparison_bar_chart():
    labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    values = [performance_metrics['accuracy'], performance_metrics['precision'], performance_metrics['recall'], performance_metrics['f1']]

    plt.figure(figsize=(8, 5))
    bars = plt.bar(labels, values, color=['blue', 'green', 'orange', 'red'])
    plt.title('Model Performance Metrics')
    plt.ylim(0, 1)  # Values are between 0 and 1
    plt.ylabel('Score')

    # Add values on top of the bars
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.02, round(yval, 4), ha='center', va='bottom', fontweight='bold')

    plt.show()

In [ ]:
plot_comparison_bar_chart()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
total_samples = len(true_labels)
tp = int(recall * (accuracy * total_samples))
fp = int((tp / precision) - tp)
fn = int((tp / recall) - tp)
tn = total_samples - (tp + fp + fn)

# Confusion matrix array
conf_matrix = np.array([[tn, fp], [fn, tp]])

# Plot confusion matrix
def plot_confusion_matrix(cm):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

# Call the function to plot the confusion matrix
plot_confusion_matrix(conf_matrix)